In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# Чтение данных из Excel
df = pd.read_excel("data.xlsx")

# Удаление пробелов в начале и в конце строк для столбцов "Источник" и "Магистраль"
df['Источник'] = df['Источник'].str.strip()
df['Магистраль'] = df['Магистраль'].str.strip()

# Создание нового столбца с комбинацией "Источник+Магистраль"
df['Источник+Магистраль'] = df['Источник'] + '+' + df['Магистраль']

# Преобразование столбца "Дата" в формат datetime
df['Дата'] = pd.to_datetime(df['Дата'], errors='coerce')

# Извлечение уникальных дат (только даты)
unique_dates = sorted(df['Дата'].dropna().dt.date.unique())

# Извлечение уникальных комбинаций "Источник+Магистраль"
unique_combinations = df['Источник+Магистраль'].dropna().unique()

In [4]:
# Создание пустых DataFrame для:
# 1. Номеров записей
# 2. Температуры наружного воздуха (Тнв)
# 3. Фактической температуры наружного воздуха (Тнвф)
work_days_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)
outer_temperature_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)
actual_temperature_df = pd.DataFrame(index=unique_dates, columns=unique_combinations)

# Заполнение DataFrame-ов
for date in unique_dates:
    # Фильтрация записей по дате
    filtered_by_date = df[df['Дата'].dt.date == date]
    for combination in unique_combinations:
        # Фильтрация по комбинации "Источник+Магистраль"
        filtered_by_combination = filtered_by_date[filtered_by_date['Источник+Магистраль'] == combination]
        if not filtered_by_combination.empty:
            # Запись номера записи
            work_days_df.at[date, combination] = filtered_by_combination.iloc[0]['№№']
            # Запись температуры наружного воздуха (из столбца с именем "Тнв")
            outer_temperature_df.at[date, combination] = filtered_by_combination.iloc[0]['Тнв']
            # Запись фактической температуры наружного воздуха (Тнвф)
            # Данные для Тнвф находятся в 14-ом столбце исходного Excel файла (индекс 13)
            actual_temperature_df.at[date, combination] = filtered_by_combination.iloc[0, 13]

In [5]:
with pd.ExcelWriter("output.xlsx", engine='xlsxwriter') as writer:
    # Лист с номерами записей
    work_days_df.to_excel(writer, sheet_name="Номера записей")
    # Лист с температурой наружного воздуха (Тнв)
    outer_temperature_df.to_excel(writer, sheet_name="Тнв")
    # Новый лист с фактической температурой наружного воздуха (Тнвф)
    actual_temperature_df.to_excel(writer, sheet_name="Тнвф")
    
    # Получаем объекты книги и листов для дальнейшего форматирования
    workbook = writer.book
    worksheet_tnv = writer.sheets["Тнв"]
    worksheet_tnvf = writer.sheets["Тнвф"]
    
    # Формат для выделения выбросов (красная заливка)
    red_format = workbook.add_format({'bg_color': '#FFC7CE'})

In [9]:
 # Функция для анализа выбросов по строкам (метод 2σ) и выделения их красной заливкой
def highlight_outliers(worksheet, df_sheet, start_row=1, start_col=1):
    """
    Проходит по строкам DataFrame и выделяет ячейки с выбросами.
    Параметры start_row и start_col задают смещение (так как первая строка – заголовок, первый столбец – индекс).
    """
    for i, date in enumerate(df_sheet.index):
        row_series = df_sheet.loc[date]
        # Приводим значения к числовому типу (нечисловые значения станут NaN)
        row_values = pd.to_numeric(row_series, errors='coerce')
        valid = row_values.dropna()
        if len(valid) < 2:
            continue  # Недостаточно данных для расчёта среднего и стандартного отклонения
        row_mean = valid.mean()
        row_std = valid.std()
        if row_std == 0:
            continue  # Если стандартное отклонение равно 0, выбросов не обнаруживаем
        # Проходим по каждому столбцу строки
        for j, col in enumerate(df_sheet.columns):
            cell_value = row_series[col]
            try:
                value = float(cell_value)
            except (ValueError, TypeError):
                continue
            if abs(value - row_mean) > 2 * row_std:
                # Записываем значение с красной заливкой
                worksheet.write(i + start_row, j + start_col, value, red_format)
    
# Применяем выделение выбросов для листа "Тнв"
highlight_outliers(worksheet_tnv, outer_temperature_df)
# Применяем выделение выбросов для листа "Тнвф"
highlight_outliers(worksheet_tnvf, actual_temperature_df)

In [15]:
# -----------------------------
# Приводим данные к числовому типу
# -----------------------------
# Если данные уже в числовом виде, этот шаг можно пропустить
outer_numeric = outer_temperature_df.apply(pd.to_numeric, errors='coerce')
actual_numeric = actual_temperature_df.apply(pd.to_numeric, errors='coerce')

# -----------------------------
# 1. Линейный график: Сравнение дневных средних температур
# -----------------------------
# Вычисляем среднее значение по строкам (по датам) для каждого датафрейма
daily_predicted = outer_numeric.mean(axis=1)
daily_actual = actual_numeric.mean(axis=1)

# Формируем DataFrame для построения графика
comparison_df = pd.DataFrame({
    'Дата': daily_predicted.index,
    'Тнв (метеорологи)': daily_predicted.values,
    'Тнвф (фактическая)': daily_actual.values
})
# Если индекс представляет даты в виде строки, приводим их к datetime:
comparison_df['Дата'] = pd.to_datetime(comparison_df['Дата'])

# Строим линейный график с Plotly Express
fig_line = px.line(
    comparison_df,
    x='Дата',
    y=['Тнв (метеорологи)', 'Тнвф (фактическая)'],
    title="Сравнение дневной средней температуры: метеорологи vs фактическая",
    labels={'value': 'Температура (°C)', 'variable': 'Тип данных'}
)

# Для сохранения графика в HTML (если требуется), раскомментируйте следующую строку:
fig_line.write_html("line_comparison.html")

#fig_line.show()


In [ ]:
# -----------------------------
# 2. Scatter plot: Сравнение по точкам для каждого значения
# -----------------------------
# Преобразуем таблицы из широкого формата в длинный (stacked) для объединения данных
stacked_predicted = outer_numeric.stack().reset_index(name='Тнв')
stacked_actual = actual_numeric.stack().reset_index(name='Тнвф')

# Объединяем данные по дате и комбинации
merged = pd.merge(stacked_predicted, stacked_actual, on=['level_0', 'level_1'])
merged = merged.rename(columns={'level_0': 'Дата', 'level_1': 'Комбинация'})
merged['Дата'] = pd.to_datetime(merged['Дата'])

# Строим scatter plot
fig_scatter = px.scatter(
    merged,
    x='Тнв',
    y='Тнвф',
    color='Комбинация',  # Если количество комбинаций велико, можно убрать этот параметр
    hover_data=['Дата', 'Комбинация'],
    title="Сравнение по точкам: метеорологи (Тнв) vs фактическая (Тнвф)",
    labels={'Тнв': 'Тнв (метеорологи)', 'Тнвф': 'Тнвф (фактическая)'}
)

# Добавляем диагональную линию y=x для наглядного сравнения
min_val = min(merged['Тнв'].min(), merged['Тнвф'].min())
max_val = max(merged['Тнв'].max(), merged['Тнвф'].max())
fig_scatter.add_trace(
    go.Scatter(
        x=[min_val, max_val],  
        y=[min_val, max_val],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='y=x'
    )
)

# Для сохранения scatter plot в HTML:
fig_scatter.write_html("scatter_comparison.html")

#fig_scatter.show()